In [2]:
%load_ext autoreload
%autoreload 2
!pip install tweepy
!pip install sklearn

import math
import gensim.models as gs
from gensim.models import KeyedVectors
import pickle as pk
import sklearn.metrics as met
import scipy.stats as stats
import numpy as np
import re
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from gensim.models.fasttext import FastText
import twitter_sentiment_dataset as tsd
import phrase2vec as p2v
from twitter_sentiment_dataset import TweetTrainingExample
import urllib.request
import io
from gensim.test.utils import datapath
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
#from model import ModelParams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\sandeep bhatula\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\sandeep bhatula\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
#w2v_path = KeyedVectors.load_word2vec_format('./pre-trained/GoogleNews-vectors-negative300.bin', binary=True)


# Setup
Load the three vector representations from files. In general, any variable with the word 'none' in it refers to Google News word2vec w/o any emoji vectors, 'ours' to Google News word2vec w/ vectors we trained, and 'theirs' to Google News word2vec with the vectors trained by Barbieri et. al.

In [4]:
w2v_path='./pre-trained/GoogleNews-vectors-negative300.bin'

#print("Check")
#print(word2vec)
in_dim = 300   # Length of word2vec vectors
out_dim = 300  # Desired dimension of output vectors
pos_ex = 4
neg_ratio = 1
max_epochs = 40
dropout = 0.0


e2v_ours_path = './pre-trained/emoji2vec_bow_desc.bin'


In [5]:

w2v = KeyedVectors.load_word2vec_format(w2v_path, binary=True)
e2v_ours = KeyedVectors.load_word2vec_format(e2v_ours_path, binary=True)



In [6]:
!pip install gensim==3.8.1
wv=Word2Vec.load(str('./pre-trained/model_swm-300-3-3'))
e2v_theirs = KeyedVectors(300)
keys=[]
#print(e2v_theirs.wv.vocab.keys())
for i in wv.wv.vocab.keys():
    if len(i)>9:
        temp=chr(int(i[4:9],16))+chr(int(i[9:14],16))
        keys.append(temp)
    else:
        temp=chr(int(i[4:9],16))
        keys.append(temp)
print(keys)
#print(wv.wv.vocab.values())
values=[]
for i in wv.wv.vocab.keys():
    values.append(np.array(wv[i]))
e2v_theirs.add(entities=keys, weights=np.array(values))

distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include\UNKNOWN
sysconfig: c:\users\sandeep bhatula\appdata\local\programs\python\python38\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\sandeep bhatula\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


['🌈', '🌉', '🍺', '🌅', '🌆', '🍻', '🌀', '🌁', '🌂', '🌃', '🇫🇷', '💘', '💙', '💔', '💕', '💖', '💗', '💐', '💑', '💒', '💓', '⏪', '👞', '🎓', '🌄', '🎑', '🎐', '👟', '☝', '⏫', '🍼', '🌜', '🌇', '💝', '💞', '💟', '💚', '💛', '💜', '🍹', '🍸', '🍱', '🌎', '🍳', '🍲', '🍵', '🌊', '🍷', '🌌', '🚿', '🚾', '😠', '👝', '🎶', '🎷', '🎴', '🎵', '㊗', '🎳', '🎰', '🎱', '📏', '㊙', '📎', '📋', '📌', '🎸', '🎹', '💧', '🇧', '👇', '👆', '💨', '👅', '🎿', '🎽', '🎾', '🎻', '🎼', '📈', '🎺', '📆', '📇', '📄', '📅', '📂', '📃', '📀', '📁', '🌾', '🌽', '😮', '🌿', '🌺', '😫', '😪', '🌻', '👐', '👑', '👒', '👓', '👔', '👕', '👖', '👗', '🔑', '🔐', '🈺', '☁', '☀', '🎒', '🎧', '🔔', '🐶', '👌', '🎥', '🇰🇷', '🐷', '🎤', '🐴', '🆎', '🈸', '🈹', '🗿', '🈴', '🈵', '🈶', '🇬🇧', '🈲', '🐳', '😧', '😦', '🌷', '😤', '🌱', '🌰', '😡', '🌲', '👚', '👛', '👜', '🌹', '🌸', '😩', '😨', '🔠', '📟', '📞', '✴', '✳', '☎', '🔤', '⚾', '⚽', '📛', '🚂', '🔦', '❌', '⬇', '⬆', '⬅', '📝', '📜', '🔥', '📚', '🔧', '🔨', '🔩', '🌚', '🅱', '🅰', '⛄', '🔬', '⛺', '⛅', '🙌', '🏯', '🏮', '🏭', '🏬', '🏫', '🏪', '📗', '🔪', '📕', '📔', '🔭', '🔮', '📑', '📐', '▶', '🇾', '📙', '📘', '🇪🇸', '🏩', '🏨', '🏧', '🏦', 

c:\users\sandeep bhatula\appdata\local\programs\python\python38\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(
C:\Users\SANDEE~1\AppData\Local\Temp/ipykernel_15796/1121666916.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  values.append(np.array(wv[i]))


In [7]:
#print(e2v_theirs['eoji1f308'])
p2v_no_emoji = p2v.Phrase2Vec(out_dim, w2v, e2v=None)
p2v_our_emoji = p2v.Phrase2Vec(out_dim, w2v, e2v=e2v_ours)
p2v_their_emoji = p2v.Phrase2Vec(out_dim, w2v, e2v=e2v_theirs)

Using stats scraped from emojitracker.com at a certain point in time, we generate two sets of emoji: the top 173 most frequently used emoji, whose usage constitutes 90% of emoji usage on Twitter, and the bottom 612 least frequently used emoji, whose usage constitutes 10% of emoji usage on Twitter.

Subsequently, 'common' will refer to the former group, while 'rare' will refer to the latter.

In [8]:
p = open('./data/tweets/frequencies_w_emoji.txt', 'r',encoding='UTF-8')
ems = p.readlines()
ems = [l.split('\t')[0] for l in ems]
p.close()
top90 = set(ems[:173])
bottom10 = set(ems[173:])
p.close()

In [9]:
def emoji_dataset_stats(tweets):
    total_tweets = len(tweets)
    total_emoji = tsd.num_tweets_with_emoji(tweets, e2v_ours, e2v_theirs, ems)
    top_90_total = tsd.num_tweets_with_emoji(tweets, set(), set(), top90)
    bottom_10_total = tsd.num_tweets_with_emoji(tweets, set(), set(), bottom10)
    return total_tweets, total_emoji, top_90_total, bottom_10_total

Statistics for the entire Twitter corpus. Counts refer to # of tweets containing emoji of a type.

In [10]:
train_tweets, test_tweets = tsd.load_training_test_sets()
print('All Tweets in corpus: %s, total emoji: %s, common emoji: %s, rare emoji: %s' % emoji_dataset_stats(tsd.get_all_examples()))
print('Training set: total tweets: %s, total emoji: %s, common emoji: %s, rare emoji: %s' % emoji_dataset_stats(train_tweets))
print('Test set: total tweets: %s, total emoji: %s, common emoji: %s, rare emoji: %s' % emoji_dataset_stats(test_tweets))

All Tweets in corpus: 64599, total emoji: 11701, common emoji: 11137, rare emoji: 1576
Training set: total tweets: 51679, total emoji: 9405, common emoji: 8950, rare emoji: 1268
Test set: total tweets: 12920, total emoji: 2296, common emoji: 2187, rare emoji: 308


In [11]:
def emoji_dataset_label_stats(tweets):
    res = dict()
    res['Positive'] = 0
    res['Negative'] = 0
    res['Neutral'] = 0
    for tweet in tweets:
        res[tweet.label] += 1/len(tweets)
    print(res)

In [12]:
emoji_dataset_label_stats(train_tweets)
emoji_dataset_label_stats(test_tweets)

{'Positive': 0.2882215213142463, 'Negative': 0.25207531105472025, 'Neutral': 0.45970316763111974}
{'Positive': 0.2876160990712081, 'Negative': 0.25069659442723424, 'Neutral': 0.4616873065016006}


## Prepare Training and Testing Vectors
Given the raw training and test tweets, calculate the vector representations for each tweet for each model.

In [13]:
train_none, train_y = tsd.prepare_tweet_vector_averages(train_tweets, p2v_no_emoji)
train_ours, _ = tsd.prepare_tweet_vector_averages(train_tweets, p2v_our_emoji)
train_theirs, _ = tsd.prepare_tweet_vector_averages(train_tweets, p2v_their_emoji)

In [14]:
test_none, test_y = tsd.prepare_tweet_vector_averages(test_tweets, p2v_no_emoji)
test_ours, _ = tsd.prepare_tweet_vector_averages(test_tweets, p2v_our_emoji)
test_theirs, _ = tsd.prepare_tweet_vector_averages(test_tweets, p2v_their_emoji)

# Classification

In [15]:
classifiers = {
    'SGD(n_iter=50) ' : SGDClassifier(max_iter=50,random_state=1234),
    'Random Forest (n_estimators=60)' : RandomForestClassifier(n_estimators=60,random_state=1234)
}

In [16]:
def train_all_with_cross_validation(train_none, train_ours, train_theirs, train_y, clf, clf_name, cv=5):
    scores_none = cross_val_score(clf, train_none, train_y, cv=cv)
    print("None: %s Train Accuracy: %0.2f (+/- %0.3f)" % (clf_name, scores_none.mean(), scores_none.std() * 2))
    
    scores_ours = cross_val_score(clf, train_ours, train_y, cv=cv)
    print("Ours: %s Train Accuracy: %0.2f (+/- %0.3f)" % (clf_name, scores_ours.mean(), scores_ours.std() * 2))
    
    scores_theirs = cross_val_score(clf, train_theirs, train_y, cv=cv)
    print("Theirs: %s Train Accuracy: %0.2f (+/- %0.3f)" % (clf_name, scores_theirs.mean(), scores_theirs.std() * 2))

In [17]:
def train_and_predict(train_data, train_y, test_data, test_y, clf):
    clf.fit(train_data, train_y)
    predictions = clf.predict(test_data)
    score = met.accuracy_score(test_y, predictions)
    f1 = met.f1_score(test_y, predictions, average='weighted')
    return predictions, score, f1

In [18]:
def train_and_predict_all(train_none, test_none, train_ours, test_ours, train_theirs, test_theirs, test_y, clf, clf_name):
    none_pred, none_acc, none_f1 = train_and_predict(train_none, train_y, test_none, test_y, clf)
    print('None: %s Test Accuracy: %0.5f, f1=%0.5f' % (clf_name, none_acc, none_f1))
    
    ours_pred, ours_acc, ours_f1 = train_and_predict(train_ours, train_y, test_ours, test_y, clf)
    ours_p = tsd.calculate_mcnemars(none_pred, ours_pred, test_y)
    print('Ours: %s Test Accuracy: %0.5f, p=%0.5f, f1=%0.5f' % (clf_name, ours_acc, ours_p, ours_f1))
    
    theirs_pred, theirs_acc, theirs_f1 = train_and_predict(train_theirs, train_y, test_theirs, test_y, clf)
    theirs_p = tsd.calculate_mcnemars(none_pred, theirs_pred, test_y)
    print('Theirs: %s Test Accuracy: %0.5f, p=%0.5f, f1=%0.5f' % (clf_name, theirs_acc, theirs_p, theirs_f1))
    
    ours_theirs_p = tsd.calculate_mcnemars(ours_pred, theirs_pred, test_y)
    print('Significance between ours and theirs: p=%0.5f' % ours_theirs_p)

## Performance on Training Set and Complete Test Set
For each classifier, we calculate the average performance of the classifier on the training set when cross validation is applied, as well as the accuracy on the complete test set.

In [19]:
for clf_name, clf in classifiers.items():
    print(clf_name)
    
    print()
    
    print('Cross Validation Accuracy on Training Set\n')
    train_all_with_cross_validation(train_none, train_ours, train_theirs, train_y, clf, clf_name, cv=5)
    
    print()
    
    print('Accuracy on Test Set\n')
    train_and_predict_all(train_none, test_none, train_ours, test_ours, train_theirs, test_theirs, test_y, clf, clf_name)
    
    print()

SGD(n_iter=50) 

Cross Validation Accuracy on Training Set

None: SGD(n_iter=50)  Train Accuracy: 0.61 (+/- 0.018)
Ours: SGD(n_iter=50)  Train Accuracy: 0.61 (+/- 0.018)
Theirs: SGD(n_iter=50)  Train Accuracy: 0.62 (+/- 0.018)

Accuracy on Test Set

None: SGD(n_iter=50)  Test Accuracy: 0.61525, f1=0.60497
Ours: SGD(n_iter=50)  Test Accuracy: 0.61780, p=0.43450, f1=0.60793
Theirs: SGD(n_iter=50)  Test Accuracy: 0.61594, p=0.80958, f1=0.60147
Significance between ours and theirs: p=0.50564

Random Forest (n_estimators=60)

Cross Validation Accuracy on Training Set

None: Random Forest (n_estimators=60) Train Accuracy: 0.58 (+/- 0.010)
Ours: Random Forest (n_estimators=60) Train Accuracy: 0.59 (+/- 0.006)
Theirs: Random Forest (n_estimators=60) Train Accuracy: 0.58 (+/- 0.008)

Accuracy on Test Set

None: Random Forest (n_estimators=60) Test Accuracy: 0.58235, f1=0.56633
Ours: Random Forest (n_estimators=60) Test Accuracy: 0.59420, p=0.00449, f1=0.58178
Theirs: Random Forest (n_estimators

In [20]:
def train_and_predict_all_on_test_subset(test_tweets, clf, clf_name):
    test_none, test_y = tsd.prepare_tweet_vector_averages(test_tweets, p2v_no_emoji)
    test_ours, _ = tsd.prepare_tweet_vector_averages(test_tweets, p2v_our_emoji)
    test_theirs, _ = tsd.prepare_tweet_vector_averages(test_tweets, p2v_their_emoji)

    train_and_predict_all(train_none, test_none, train_ours, test_ours, train_theirs, test_theirs, test_y, clf, clf_name)

In [21]:
emoji_test_tweets = tsd.get_tweets_with_emoji(test_tweets, e2v_ours, e2v_theirs, ems)
emoji_test_tweets_top90 = tsd.get_tweets_with_emoji(test_tweets, set(), set(), top90)
emoji_test_tweets_bottom10 = tsd.get_tweets_with_emoji(test_tweets, set(), set(), bottom10)

## Test Subset - All Tweets with Emoji
For each classifier, we calculate the accuracy on the subset of test examples that contain emoji.

In [22]:
for clf_name, clf in classifiers.items():
    print(clf_name)
    train_and_predict_all_on_test_subset(emoji_test_tweets, clf, clf_name)
    print()

SGD(n_iter=50) 
None: SGD(n_iter=50)  Test Accuracy: 0.51045, f1=0.52333
Ours: SGD(n_iter=50)  Test Accuracy: 0.61237, p=0.00000, f1=0.60779
Theirs: SGD(n_iter=50)  Test Accuracy: 0.58057, p=0.00000, f1=0.59213
Significance between ours and theirs: p=0.00180

Random Forest (n_estimators=60)
None: Random Forest (n_estimators=60) Test Accuracy: 0.47997, f1=0.49073
Ours: Random Forest (n_estimators=60) Test Accuracy: 0.58145, p=0.00000, f1=0.57551
Theirs: Random Forest (n_estimators=60) Test Accuracy: 0.52831, p=0.00001, f1=0.54418
Significance between ours and theirs: p=0.00003



## Test Subset - All Tweets with Common Emoji
For each classifier, we calculate the accuracy on the subset of test examples that contain common (Top 90%) emoji.

In [23]:
for clf_name, clf in classifiers.items():
    print(clf_name)
    train_and_predict_all_on_test_subset(emoji_test_tweets_top90, clf, clf_name)
    print()

SGD(n_iter=50) 
None: SGD(n_iter=50)  Test Accuracy: 0.51212, f1=0.52588
Ours: SGD(n_iter=50)  Test Accuracy: 0.61774, p=0.00000, f1=0.61200
Theirs: SGD(n_iter=50)  Test Accuracy: 0.58390, p=0.00000, f1=0.59546
Significance between ours and theirs: p=0.00110

Random Forest (n_estimators=60)
None: Random Forest (n_estimators=60) Test Accuracy: 0.48194, f1=0.49364
Ours: Random Forest (n_estimators=60) Test Accuracy: 0.59214, p=0.00000, f1=0.58457
Theirs: Random Forest (n_estimators=60) Test Accuracy: 0.53132, p=0.00001, f1=0.54766
Significance between ours and theirs: p=0.00000



## Test Subset - All Tweets with Rare Emoji
For each classifier, we calculate the accuracy on the subset of test examples that contain rare (Bottom 10%) emoji.

In [ ]:
for clf_name, clf in classifiers.items():
    print(clf_name)
    train_and_predict_all_on_test_subset(emoji_test_tweets_bottom10, clf, clf_name)
    print()

SGD(n_iter=50) 
None: SGD(n_iter=50)  Test Accuracy: 0.45779, f1=0.45660
Ours: SGD(n_iter=50)  Test Accuracy: 0.56494, p=0.00341, f1=0.56746
Theirs: SGD(n_iter=50)  Test Accuracy: 0.54545, p=0.00238, f1=0.55093
Significance between ours and theirs: p=0.52243

Random Forest (n_estimators=60)
